# 목표

- 관세청 영문 문장 데이터 추가

# Library

In [ ]:
# visualization
import matplotlib
matplotlib.rcParams['axes.unicode_minus'] = False
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
fe = fm.FontEntry(
    fname=r'.venv/Lib/site-packages/matplotlib/mpl-data/fonts/ttf/NanumGothic.ttf', # ttf 파일이 저장되어 있는 경로
    name='NanumBarunGothic')                        # 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 10, 'font.family': 'NanumBarunGothic'}) # 폰트 설정
plt.rc('font', family='NanumBarunGothic')
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import missingno as msno

In [ ]:
import os
import re
import json
import yaml
import time
import numpy as np
import pandas as pd

from tqdm import tqdm
from pprint import pprint

from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.common.by import By

# Data Load

In [ ]:
text_data = pd.read_csv('./data/영문_텍스트.csv', dtype=str)
statistics_data = pd.read_csv('./data/통계청.csv', dtype=str)
customs_data = pd.read_csv('./data/관세청.csv', dtype=str)

In [ ]:
text_data_copy = text_data.copy()
statistics_data_copy = statistics_data.copy()
customs_data_copy = customs_data.copy()

In [ ]:
display(text_data_copy.info())
display(statistics_data_copy.info())
display(customs_data_copy.info())

In [ ]:
customs_data_copy['HS부호'].apply(lambda x: x[:4])

In [ ]:
customs_data_copy['HS부호_앞4자리'] = customs_data_copy['HS부호'].apply(lambda x: x[:4])

In [ ]:
print(customs_data_copy.shape[0])
print(customs_data_copy['HS부호_앞4자리'].nunique())

# 앞 4자리로 기준 잡으면 됨 (앞4자리, 앞6자리 비교했는데 앞4자리가 공통되는 기준임)

In [ ]:
customs_data_copy['HS부호_앞4자리'].unique()

In [ ]:
customs_data_copy[customs_data_copy['HS부호_앞4자리'] == '0101']

- 0101, 010121, 0101211000의 설명은 같음

In [ ]:
# -------------------- 함수 모음 --------------------
## 스크롤 끝까지
def scroll(driver):
    scroll_location = driver.execute_script('return document.body.scrollHeight')
    while True:
        driver.execute_script('window.scrollTo(0,document.body.scrollHeight)')
        time.sleep(2)
        scroll_height = driver.execute_script('return document.body.scrollHeight')
        if scroll_location == scroll_height:
            break
        else:
            scroll_location = driver.execute_script('return document.body.scrollHeight')
    driver.implicitly_wait(3)


## 스크롤 한 번만
def scroll_one(driver):
    driver.execute_script('window.scrollTo(0,document.body.scrollHeight)')
    time.sleep(1)
    driver.implicitly_wait(3)

## 스크롤 위로
def scroll_top(driver):
    driver.execute_script('window.scrollTo(0, 0)')
    time.sleep(1)
    driver.implicitly_wait(3)


## 페이지 내의 특정 스크롤 끝까지
def scroll_element(driver, cond):
    element = driver.find_element(By.CSS_SELECTOR, cond)
    scroll_location = driver.execute_script('return arguments[0].scrollHeight', element)
    while True:
        driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', element)
        time.sleep(2)
        scroll_height = driver.execute_script('return arguments[0].scrollHeight', element)
        if scroll_location == scroll_height:
            break
        else:
            scroll_location = driver.execute_script('return arguments[0].scrollHeight', element)
    driver.implicitly_wait(3)

In [ ]:
# -------------------- 크롬 준비 --------------------
options = Options()
options.add_experimental_option("detach", True) # 브라우저 꺼짐 방지
options.add_argument('--start-maximized') # 브라우저 최대화 (지정도 가능: --window-size=1920, 1080)

service = Service(excutable_path=ChromeDriverManager().install())

In [23]:
driver = webdriver.Chrome(service=service, options=options)
url = 'https://unipass.customs.go.kr/clip/index.do'
driver.get(url=url)
time.sleep(0.5)

config = {
    'main_kor_dsc': dict(),
    'main_eng_dsc': dict(),
    'all_kor_dsc': dict(),
    'all_eng_dsc': dict(),
}
keywords = customs_data_copy['HS부호_앞4자리'].unique().tolist()

# 세계HS 탭 클릭
btn = driver.find_element(By.XPATH, '//*[@id="topmenu"]/ul/li[3]')
btn.click()
time.sleep(0.5)

# 관세율표 클릭
btn = driver.find_element(By.XPATH, '//*[@id="leftmenu"]/ul/li[1]/ul/li[3]')
btn.click()

for keyword in tqdm(keywords):
    time.sleep(0.5)

    # 검색어 입력
    inp = driver.find_element(By.XPATH, '//*[@id="searchVal"]')
    inp.send_keys(f'{keyword}')
    time.sleep(0.5)

    # 검색 버튼 클릭
    btn = driver.find_element(By.CLASS_NAME, 'btn_inlinesearch')
    btn.click()
    time.sleep(0.5)

    # 핵심 국문 텍스트 수집
    try:
        parse = driver.find_element(By.XPATH, '//*[@id="tblLstBody"]/tr[1]/td[4]').text
        config['main_kor_dsc'][f'{keyword}'] = parse
    except Exception as NoSuchElementException:
        config['main_kor_dsc'][f'{keyword}'] = np.nan

    # 핵심 영문 텍스트 수집
    try:
        parse = driver.find_element(By.XPATH, '//*[@id="tblLstBody"]/tr[1]/td[5]').text
        config['main_eng_dsc'][f'{keyword}'] = parse
    except Exception as NoSuchElementException:
        config['main_eng_dsc'][f'{keyword}'] = np.nan

    scroll(driver) # 전체 페이지 스크롤 끝까지 아래로

    # 국문 텍스트 전체 수집
    scroll_element(driver, '#divLft_tab4') # 특정 페이지 스크롤 끝까지
    parse = driver.find_element(By.XPATH, '//*[@id="divLft_tab4"]').text
    config['all_kor_dsc'][f'{keyword}'] = parse

    # 영문 버튼 클릭
    btn = driver.find_element(By.XPATH, '//*[@id="tab4"]/div/div[1]/span/button[2]')
    btn.click()
    time.sleep(0.5)

    # 영문 텍스트 전체 수집
    scroll_element(driver, '#divLft_tab4') # 특정 페이지 스크롤 끝까지
    parse = driver.find_element(By.XPATH, '//*[@id="divLft_tab4"]').text
    config['all_eng_dsc'][f'{keyword}'] = parse

    scroll_top(driver) # 전체 페이지 스크롤 끝까지 위로

    # 검색어 리셋
    inp = driver.find_element(By.XPATH, '//*[@id="searchVal"]')
    inp.clear()

driver.quit()

# json 형식으로 데이터 저장
with open('./data/customs_crawl_data.json', 'w', encoding='utf-8-sig') as file:
    json.dump(config, file, ensure_ascii=False, indent=4)

100%|██████████| 1352/1352 [3:59:53<00:00, 10.65s/it]  


- 검색되지 않는 부호가 있음 (Ex. 0050, 0007)
    - 이상치인가?

In [48]:
keys = config['all_eng_dsc'].keys()
values0 = config['main_kor_dsc'].values()
values1 = config['main_eng_dsc'].values()
values2 = config['all_kor_dsc'].values()
values3 = config['all_eng_dsc'].values()

data = {
    'HS부호_앞4자리': keys,
    '핵심_한글설명': values0,
    '핵심_영어설명': values1,
    '전체_한글설명': values2,
    '전체_영어설명': values3,
}
crawl_data = pd.DataFrame(data)

In [50]:
crawl_data.head()

,HS부호_앞4자리,핵심_한글설명,핵심_영어설명,전체_한글설명,전체_영어설명
0,0101,살아 있는 말ㆍ당나귀ㆍ노새ㆍ버새,"Live horses, asses, mules and hinnies.",01.01 - 살아 있는 말ㆍ당나귀ㆍ노새ㆍ버새(+)\n- 말\n0101.21 - 번...,"01.01 Live horses, asses, mules and hinnies (+..."
1,0102,살아 있는 소,Live bovine animals.,01.02 - 살아 있는 소(+)\n- 축우(畜牛)\n0102.21 - 번식용\n0...,01.02 Live bovine animals (+).\n- Cattle :\n01...
2,0001,NaN,NaN,조회결과가 존재하지 않습니다.,조회결과가 존재하지 않습니다.
3,0103,살아 있는 돼지,Live swine.,01.03 - 살아 있는 돼지(+)\n0103.10 - 번식용\n- 기타\n0103...,01.03 - Live swine (+).\n0103.10 - Pure bred b...
4,0104,살아 있는 면양과 염소,Live sheep and goats.,01.04 - 살아 있는 면양과 염소\n0104.10 - 면양\n0104.20 - ...,01.04 - Live sheep and goats.\n0104.10 - Sheep...


In [49]:
crawl_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1352 entries, 0 to 1351
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   HS부호_앞4자리  1352 non-null   object
 1   핵심_한글설명    1229 non-null   object
 2   핵심_영어설명    1229 non-null   object
 3   전체_한글설명    1352 non-null   object
 4   전체_영어설명    1352 non-null   object
dtypes: object(5)
memory usage: 52.9+ KB


In [67]:
crawl_data[crawl_data['핵심_한글설명'].isnull()]

,HS부호_앞4자리,핵심_한글설명,핵심_영어설명,전체_한글설명,전체_영어설명
2,0001,NaN,NaN,조회결과가 존재하지 않습니다.,조회결과가 존재하지 않습니다.
14,0002,NaN,NaN,조회결과가 존재하지 않습니다.,조회결과가 존재하지 않습니다.
15,0000,NaN,NaN,조회결과가 존재하지 않습니다.,조회결과가 존재하지 않습니다.
20,0003,NaN,NaN,조회결과가 존재하지 않습니다.,조회결과가 존재하지 않습니다.
21,0030,NaN,NaN,조회결과가 존재하지 않습니다.,조회결과가 존재하지 않습니다.
...,...,...,...,...,...
1304,0930,NaN,NaN,조회결과가 존재하지 않습니다.,조회결과가 존재하지 않습니다.
1312,0094,NaN,NaN,조회결과가 존재하지 않습니다.,조회결과가 존재하지 않습니다.
1318,0095,NaN,NaN,조회결과가 존재하지 않습니다.,조회결과가 존재하지 않습니다.
1319,0950,NaN,NaN,조회결과가 존재하지 않습니다.,조회결과가 존재하지 않습니다.


In [66]:
any(crawl_data[crawl_data['핵심_한글설명'].isnull()].index == crawl_data[crawl_data['핵심_영어설명'].isnull()].index)

True

In [70]:
any(crawl_data[crawl_data['전체_한글설명'] == '조회결과가 존재하지 않습니다.'].index == crawl_data[crawl_data['전체_영어설명'] == '조회결과가 존재하지 않습니다.'].index)

True

In [81]:
temp = []
for idx in range(0, crawl_data.shape[0]):
    if crawl_data.iloc[idx, 4] == ' ': # ''
        temp.append(idx)
print(len(temp))

0


In [82]:
crawl_data.to_csv('./data/customs_crawl_data.csv', index=False, encoding='utf-8-sig')

# 정규식으로 처리

In [85]:
crawl_data = pd.read_csv('./data/customs_crawl_data.csv', dtype=str)

In [87]:
crawl_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1352 entries, 0 to 1351
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   HS부호_앞4자리  1352 non-null   object
 1   핵심_한글설명    1229 non-null   object
 2   핵심_영어설명    1229 non-null   object
 3   전체_한글설명    1352 non-null   object
 4   전체_영어설명    1352 non-null   object
dtypes: object(5)
memory usage: 52.9+ KB


In [97]:
keys = crawl_data['HS부호_앞4자리'].tolist()

## 전체_* 컬럼 예시

1. 조회결과가 아닌 경우, 두번째 파트 윗부분 버리기

In [106]:
keys[396]

'2935'

In [107]:
text = crawl_data.loc[396, '전체_한글설명']
print(text)

29.35 – 술폰아미드*
2935.10 - 엔-메틸과불화옥탄 술폰아미드
2935.20 - 엔-에틸과불화옥탄 술폰아미드
2935.30 - 엔-에틸-엔-(2-히드록시에틸)과불화옥탄 술폰아미드
2935.40 - 엔-(2-히드록시에틸)-엔-메틸과불화옥탄 술폰아미드
2935.50 - 그 밖의 과불화옥탄 술폰아미드
2935.90 - 기타
술폰아미드는 (R1SO2NR2R3)의 일반식을 가지고 있으며 여기에서 R1은 이산화황(SO2) 그룹에 직접 붙어 있는 탄소 원자를 갖고 있는 여러 가지 복합물의 유기기(有機基 : organic radical)이며 R2와 R3는 수소나 그 밖의 원자이거나 여러 가지 복합물의 무기기(無機基)나 유기기(이중 결합이나 고리를 포함한다)이다. 강한 살세균제로서 의약용에 많이 사용한다. 이 호에는 특히 다음의 것을 포함한다.
(1) N-알킬과불화옥탄 술폰아미드* : 예로서는 N-메틸과불화옥탄 술폰아미드나 N-에틸-N-(2-히드록시에틸)과불화옥탄 술폰아미드를 들 수 있다. 이 화학품들은 분해되어 과불화옥탄 술포네이트(PFOS)를 형성한다(제2904호, 제2922호, 제2923호, 제3808호와 제3824호도 참조).
(2) 오르토톨루엔술폰아미드
(3) 오르토술파모일벤조산
(4) 파라술파모일벤질아민
(5) 파라아미노벤젠술폰아미드(H2NC6H4SO2NH2)(술파닐아미드)*
(6) 파라아미노벤젠술폰아세트아미드
(7) 구연산 실데나필(비아그라)
(8) 술파피리딘(INN)이나 파라아미노벤젠술폰아미도피리딘
(9) 술파디아진(INN)이나 파라아미노벤젠술폰아미도피리미딘
(10) 술파메라진(INN)이나 파라아미노벤젠술폰아미도메틸피리미딘
(11) 술파티오우레아(INN)나 파라아미노벤젠술폰아미도티오우레아
(12) 술파티아졸(INN)이나 파라아미노벤젠술폰아미도티아졸
(13) 술폰아미드의 염화물(염소원자가 직접 질소와 결합하여 있는지에는 상관없다)(예: “클로라민”으로 알려져 있는 술폰클로라미드나 엔-클로로술폰아미드 ; “클로로티아지드”나 6-클로로-7-술파모일벤조-1,

## 시작!

In [ ]:
def part_split(config, part_2_config):
    for key in tqdm(keywords):
        text = config[key]

        # 텍스트 값 확인
        if text == '조회결과가 존재하지 않습니다.':
            part_2_config[key] = text

        else:
            # # 기호 제거 (◦, *)
            # if '◦\n' in text:
            #     text = re.sub(r'\n\◦\n\◦ \◦\n[\s\S]*', '', text).strip()
            # if '*\n' in text:
            #     text = re.sub(r'\n\*\n\* \*\n[\s\S]*', '', text).strip()

            # # 첫번째 파트 추출
            # pattern = r'\d{2}\.\d{2} [^\n]+'
            # part_1 = re.search(pattern, text).group()
            
            # part_1_config[key] = part_1

            # # 첫번째 파트 제거
            # pattern = r'\d{2}\.\d{2} [^\n]+'
            # text = re.sub(
            #     pattern=pattern,
            #     repl='',
            #     string=text,
            #     count=1,
            #     flags=re.MULTILINE
            # ).strip()

            # 두번째 파트 추출 (nnnn.nn 및 - 지우기)
            pattern = r'\d{4}\.\d{2} [^\n]+'
            text = re.sub(
                pattern=pattern, 
                repl='', 
                string=text
            ).strip()

            pattern = r'^- .*$'
            part_2 = re.sub(
                pattern=pattern, 
                repl='', 
                string=text, 
                flags=re.MULTILINE
            ).strip()

            part_2_config[key] = part_2

    return part_2_config